In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
import pandas as pd
import random
from datetime import datetime, date
import datetime 
import time
import os
import csv
from concurrent.futures import ThreadPoolExecutor
import openpyxl

In [2]:
options= webdriver.ChromeOptions()
# options.add_argument('--headless')  #後臺運行(看執行結果)
# options.add_argument('--no-sandbox') 
# options.add_argument('--disable-gpu') #ban GPU加速
# options.add_argument('--disable-dev-shm-usage') #ban內存
#完成設定
driver = uc.Chrome(options=options)
driver.set_window_size(700, 1000)
driver.get('https://www.chinatimes.com/?chdtv')

In [3]:
url = []
Category = []
NEWS_category = []
news_url = []
title =  []
publish_time = []
content = []

data = {
    'Link': url,
    'Category': Category,
   }

compare_news_time = datetime.date.today().strftime('%Y/%m/%d')   #這邊改過
print(compare_news_time)
now = datetime.date.today().strftime('%Y-%m-%d')

2025/03/02


In [4]:
category_bar = driver.find_elements(By.CLASS_NAME,'main-nav-item') #這是index 列


for index in category_bar[1:21]:
    a_tag = index.find_element(By.TAG_NAME, 'a')  #這個是index的href
    time.sleep(0.01)
    Category.append(index.text)   #這個是index 內的category名
    if a_tag:
        href = a_tag.get_attribute('href')
        url.append(href)

In [5]:
def process_news_page(href):
    driver = webdriver.Chrome()  # 多線程 
    driver.get(href)
    time.sleep(5)

    try:
        news_category = driver.find_elements(By.TAG_NAME, 'span')[1].get_attribute("innerText")
        news_title = driver.find_element(By.CLASS_NAME, 'article-title').text
        news_date = driver.find_element(By.CLASS_NAME, 'date').text
        article_bodies = driver.find_elements(By.CLASS_NAME, 'article-body')
        news_content = ""
        for body in article_bodies:
            paragraphs = body.find_elements(By.TAG_NAME, 'p')
            for paragraph in paragraphs:
                news_content += paragraph.text + "\n"
        
        news_data = {
            'Link': href,
            'Category': news_category,
            'Title': news_title,
            'Publish_time': news_date,
            'Text': news_content
        }
        
        print(f"category: {news_category}")
        print(f"Title: {news_title}")
        print(f"Date: {news_date}")
        print(f"Content: {news_content}")
        
        return news_data

    except Exception as e:
        print(f"Error getting news details from {href}: {e}")
        return None
     
    finally:
        driver.quit()  


all_news_data = []


for get_url in data['Link']:
    print(get_url) 

    try:
        driver.get(get_url)
        time.sleep(1)

        try:
            more = driver.find_element(By.CLASS_NAME, 'more')  # 找 "more" 按钮
            print(more.text)
            more.click()  # 按 "more" 按钮
            time.sleep(10)  

            while True:
                time.sleep(5)
                news_pages = driver.find_elements(By.CLASS_NAME, 'articlebox-compact')  # 放新聞的box
                news_urls = []
                all_today_news = True  # 標記是否所有news都是今天的

                for get_news in news_pages:
                    news_time = get_news.find_element(By.CLASS_NAME, 'date').text

                    if news_time == compare_news_time:
                        page_links = get_news.find_elements(By.TAG_NAME, 'h3')

                        for page_link in page_links:
                            a_tags = page_link.find_elements(By.TAG_NAME, 'a')
                            for a_tag in a_tags:
                                href = a_tag.get_attribute('href')

                                if len(href) <= 300:  # 判斷廣告
                                    news_urls.append(href)
                                    print('news link' + ': ' + href)
                    else:
                        all_today_news = False
                        break

                
                with ThreadPoolExecutor(max_workers=10) as executor:  #lag就縮減
                    
                    news_data_list = list(executor.map(process_news_page, news_urls))
                    
                    all_news_data.extend(news_data_list)

                if not all_today_news:
                    print("No more news from today on this page.")
                    break  

                try:
                    next_page = driver.find_element(By.XPATH, '//a[contains(@class, "page-link") and text()="下一頁"]')
                    next_page.click()
                    time.sleep(10)
                except Exception as e:
                    print('Cannot click next page: ')
                    break  # 

        except Exception as e:
            print(f"Could not click 'more' button on {get_url}")

    except Exception as e:
        print(f"Error accessing {get_url}")
        continue



https://www.chinatimes.com/search/%E6%BE%A4%E5%80%AB%E6%96%AF%E5%9F%BA?ctrack=pc_main_search_p02
Could not click 'more' button on https://www.chinatimes.com/search/%E6%BE%A4%E5%80%AB%E6%96%AF%E5%9F%BA?ctrack=pc_main_search_p02
https://www.chinatimes.com/realtimenews/?chdtv
Could not click 'more' button on https://www.chinatimes.com/realtimenews/?chdtv
https://www.chinatimes.com/politic/?chdtv
更多
政治 最新新聞
news link: https://www.chinatimes.com/realtimenews/20250302002123-260407
news link: https://www.chinatimes.com/realtimenews/20250302002098-260407
news link: https://www.chinatimes.com/realtimenews/20250302002048-260407
news link: https://www.chinatimes.com/realtimenews/20250302002030-260407
news link: https://www.chinatimes.com/realtimenews/20250302001996-260407
news link: https://www.chinatimes.com/realtimenews/20250302001992-260407
news link: https://www.chinatimes.com/realtimenews/20250302001969-260407
news link: https://www.chinatimes.com/realtimenews/20250302001929-260407
news link

category: 政治
Title: 急診壅塞...資深消防員吐真實心聲「給2解方」 洪孟楷不忍了
Date: 2025/03/02
Content: 醫院急診壅塞未解，引爆醫護怒火，病患們「一床難求」也讓醫護人員們幾近崩潰。國民黨立委洪孟楷今（2日）在臉書說，急診量能要對症下藥，政府高官們聽聽第一線人員的心聲吧，他並分享一位來自直轄市的資深消防人員，就本身在第一線目擊的情況後，所提供的2大建議，包含趁此時執行救護車急診就醫分級收費制度、推敬老卡、身心障礙人士、其它相關有點數之民眾，點數可用於計程車急診使用。

洪孟楷在臉書指出，現在需要的絕非用綠營高官們的大腦袋，去做造假作秀的急診室視察，而是應該真正聽取第一線人員的需求，給予支持、滿足；他喊話政府，公務人員大多數都非常優秀，相信專業、別再關冷氣房裡網軍治國，第一線的腦袋們不只不小，更比大官們更能了解問題的核心和解方。他並分享某直轄市的一位資深消防人員提供的建議。

該位消防員建議政府，第一，趁此時執行救護車急診就醫分級收費制度，當前因救護車的濫用，目前消防局救護車不是派不出來，就是要以更遠的距離來派遣救護車，因此造成消防局救災人力不足（皆長時間在執行救護勤務)）、各大醫院急診室塞爆、影響真正需要救護車的民眾（例如OHCA、急症病患）、各大醫院ICU、病床數不足（大部份來自急診重症）。


該位消防員提出第二點為，推敬老卡、身心障礙人士、其它相關有點數之民眾，點數可用於計程車急診使用，除了可增加計程車司機的收入及更多人肯加入這個行業，也能避免非重症病人叫救護車排擠重症病人使用，也可訓練司機基本救護常識及消毒（可安排在離院時消毒），車上也可配備AED，提高AED的使用率。

category: 政治
Title: 澤倫斯基和川普在白宮大吵有陷阱？《百靈果》Ken爆內幕
Date: 2025/03/02
Content: 烏克蘭總統澤倫斯基2月28日與美國總統川普、副總統范斯會面，原本預計將簽署涵蓋稀土在內的礦產協議，未料3人就烏俄戰爭問題爆發激烈口角衝突，場面一度緊張，也引發全球關注。對此，Podcast節目《百靈果News》主持人楊超然（Ken）2日就表示，「我看了澤倫斯基去白宮跟川普和范斯的會談，我覺得澤倫斯基被偷襲了，這其實就是美國設計好的一個陷阱，來測試烏克蘭的底線」，引發討論。

Ken2日在YouTub

category: 生活
Title: 苗栗露營場控管汙水新制上路 違規最高罰300萬元
Date: 2025/03/02
Content: 苗栗縣好山好水是國內愛好露營民眾經常選擇露營的去處，縣內大小露營場超過300座。環境部去年元月修正公告規範水污染管制區內的露營場，應設置污水處理設施妥善收集與處理污水，並從今年3月1日起實施，縣府環保局特別連辦5場說明會，請露營場業者確實配合改善，違規者將面臨新台幣3萬元至300萬元罰鍰。

目前苗栗縣露營場超過300座，多數集中在泰安鄉及南庄鄉等地山區，其餘鄉鎮也有露營場，多位在偏遠山區為主，環境部為防範露營活動產生的生活污水造成環境及水源水質污染，於去年元月16日修正公告「禁止足使水污染行為」，規範水污染管制區內的露營場，應設置污水處理設施妥善收集與處理污水，確保放流水質符合標準。並提供業者一年緩衝期，這項新規範將從今年3月1日正式施行。

環保局為協助業者適應新法規，從2月24日、25日、26日分別舉辦露營區管理暨飲用水安全宣導會共5場次，向業者說明相關規範與改善措施，邀請苗栗縣露營觀光產業發展協會及許多露營場業者出席。

環保局長陳華盛表示，縣內多數露營場雖未達水污染防治法列管事業規模，但考量許多露營場址設置於山區，露營活動產生的沖洗式廁所排水及生活雜排水若未妥善收集與處理，可能對環境及水源地造成污染，因此依水污染防治法第30條第1項第5款修正公告，要求露營場須設置污水處理設施，或委託合法處理業者代為處理，違規者將面臨新台幣3萬元至300萬元罰鍰。

category: 生活
Title: 41歲方大同病逝！醫揭「反覆氣胸」原因 1習慣風險增20倍
Date: 2025/03/02
Content: 41歲金曲歌王方大同不敵病魔病逝，享年41歲，生前他曾出現氣胸等症狀，數度進出醫院，一度消失樂壇。醫師黃軒指出，20～40歲年輕人反覆發生氣胸，常見原因有原發性、次發性自發性氣胸、遺傳與結締組織疾病、自體免疫與發炎性疾病、外傷與特殊因素等5種，其中有吸菸習慣的人，氣胸風險增加20倍，尤其是有肺大泡或肺結構異常者。

氣胸也就是爆肺，是指肺臟和胸壁之間的「肋膜腔」出現氣體不正常進入其中，肺無法順利擴張，導致不能正常呼吸。黃軒在臉書表示，原發性自發性氣胸常見於瘦高的年輕男性，尤其是20～30歲，與肺大泡破裂有關，首次氣胸後1到

category: 生活
Title: 憂孩童過敏！民怨公費打嘸其他疫苗 北市衛生局回應了
Date: 2025/03/02
Content: 台北市每年10月會陸續安排校園接種公費流感疫苗，但有家長反映，孩子因曾接種雞胚胎蛋培養流感疫苗引發嚴重過敏反應，得自行至合約院所接種，到了現場才得知已無公費細胞培養流感疫苗可接種，只得自費施打。議員建議，衛生局應事先統計對雞胚胎蛋培養流感疫苗有嚴重反應史的人數，並保留一定數量的公費細胞培養流感疫苗供其接種，保障市民權益。衛生局回應，疾管署於2024年的流感疫苗接種計畫並無相關規定，但若學生有特殊需求可向學校反映，會協助轉介接種。

民進黨議員張文潔表示，有家長反應，因學校集中接種的公費流感疫苗是隨機廠牌，但孩子曾接種雞胚胎蛋培養流感疫苗引發嚴重過敏反應，因此無法配合在校集中接種。但令民眾錯愕的是，當孩子收到補接種通知單、自行到配合的院所接種時，卻被告知已無公費細胞培養流感疫苗可接種，只得改自費施打。

張文潔指出，根據疾管署統計，全台去年10月1日至今年2月19日止，總計接獲61件流感疫苗嚴重不良事件通報。而美國疾病管制中心早就提出，曾對雞胚胎蛋流感疫苗的成分產生嚴重過敏者，不應再接種含有該成分的疫苗；台灣疾管署也早有相關建議，包括已知對疫苗成分過敏或過去注射曾發生嚴重不良反應者，不予接種。

張文潔建議，北市應事先統計曾對雞胚胎蛋培養流感疫苗有嚴重反應的人數，並事先保留一定數量的公費細胞培養流感疫苗供其接種，同時建檔，未來到了流感疫苗施打期間再通知個案至指定單位接種公費細胞培養流感疫苗。

衛生局疾管科長張惠美表示，疫苗接種是依據醫學相關實證及專家會議討論後訂定，疾管署於2024年的流感疫苗接種計畫並無規定必須事先調查統計並預留其他疫苗。且農曆春節前後，北市流感疫苗各廠牌均有庫存，如果學生有特殊需求可事前向學校反映，會協助轉介至有庫存疫苗的院所接種。

張惠美補充，大部分流感疫苗的病毒是來自於雞胚蛋培養，傳統上認為疫苗中可能含有雞蛋蛋白成分，曾建議有雞蛋過敏史者接種後需觀察一段時間，確定未產生過敏反應。但近期研究顯示，流感疫苗製程進步，內含的雞蛋蛋白成分已極少，雞蛋過敏者接種雞胚蛋培養的流感疫苗並不會增加過敏反應發生率，因此國際上皆建議雞蛋過敏者可安心照一般流程接種流感疫苗。若仍出現過敏反應，則應由醫師評估給予施打建議。


category: 生活
Title: 新北勞工局提醒：外籍移工符合條件可領「舊制退休金」
Date: 2025/03/02
Content: 《勞工退休金條例》新制退休金制度實施後，雇主自2005年7月1日聘用本國籍勞工，依法須每月提撥6％新制退休金至個人專戶。但外國專業技術及中階技術人力仍適用《勞基法》舊制勞工退休金制度，雇主須在台灣銀行信託部開立勞退舊制專戶，依勞工每月薪資總額2％至15％按月提撥至專戶。

新北市勞工局長陳瑞嘉表示，依《勞基法》第53條規定，勞工在同一事業單位工作年資達15年且年滿55歲、年資達10年且年滿60歲時，或者年資達25年，即可向雇主申請退休並請求退休金。專業技術及中階技術移工在台無工作年限限制，未來仍可能符合退休條件，提醒雇主須依法提撥退休準備金。

如欲瞭解勞退新、舊制法令及實務操作，勞工局將於12日與勞動部合辦「勞工退休金制度及法令說明會」，歡迎轄內事業單位至新北勞動雲網站報名參加。

category: 生活
Title: 不是流感！喉嚨超痛又發燒 醫快篩：真的中獎了
Date: 2025/03/02
Content: 小兒科醫師羅士軒分享，自己近日喉嚨很痛、有點微發燒、酸痛感，懷疑得了流感或A型鏈球菌感染，因此快篩，結果真的中獎，是A型鏈球菌感染，必須吃抗生素治療。他指出A型鏈球菌偶而會造成嚴重感染，甚至產生致命的鏈球菌毒性休克症候群。

羅士軒近日在臉書PO出一張快篩的照片，表示這個不是病人的快篩，而是他自己的快篩，沒想到真的中了。表示因為喉嚨很痛、有點微燒、痠痛感，但沒有看到腸病毒的潰瘍，懷疑會不會是染流感或A型鏈球菌，至少這兩個感染有藥物可以治療，使用抗流感藥或抗生素，早點快篩驗出來早點讓症狀緩和，結果真的猜中是「A型鏈球菌」感染。

羅士軒表示，A型鏈球菌跟經常聽到的肺炎鏈球菌是不一樣的細菌，只是都屬於鏈球菌家族。A型鏈球菌不管大人或小孩都可能感染。最常見的症狀是咽喉炎及扁桃腺炎，可能伴隨一般感冒輕微咳嗽、流鼻水。「猩紅熱」則是另一種症狀表現，除了發燒、扁桃腺炎之外，還會出現草莓舌、摸起來如砂紙般粗糙的皮疹。

羅士軒說，A型鏈球菌偶而會造成嚴重感染，包括蜂窩型組織炎、壞死性筋膜炎，也就是日本流行的食肉菌，以及風溼性心臟病、急性腎絲球腎炎，甚至會產生致命的鏈球菌毒性休克症候群（STSS）。

羅士軒指出，如果是A型

...

category: 體育
Title: MLB》日籍強投達比修有展現三振功力！ 模擬賽3局送6K
Date: 2025/03/02
Content: #Padres starting pitcher. Yu Darvish pitches 3 innings again CWS minor prospects on Field 2 in Peoria. Yu had 6 K's on 49 pitches. #GoPadres2025 pic.twitter.com/NbT7l3pBPq
聖地牙哥教士隊日籍強投達比修有，今天（2日）進行模擬比賽（Live BP），對芝加哥白襪隊小聯盟打者，主投3局雖然失4分，卻也展現三振功力送出6K。

根據日媒《日刊體育》報導，達比修共投3局、49球，被擊出5支安打，包括1支全壘打，失4分，送出6次三振，沒有保送。

本場比賽中，白襪隊內野手達爾貝（Bobby Dalbec）對達比修敲出全壘打。達爾貝過去曾效力紅襪隊5個球季，2021年敲出生涯最多的25轟，大聯盟生涯47轟，本季與白襪隊簽下小聯盟合約，並受邀參加大聯盟春訓。

今年是達比修在大聯盟的第13個球季，去年經歷左腹股溝以及手肘傷勢，以及處理「家中私事」影響，整季出賽16場，拿下7勝3敗，防禦率3.31、78次三振成績。生涯累積110勝、88敗，防禦率3.58、三振數2007。

category: 體育
Title: TPBL》飛馳末節狂飆21分 全場37分率領特攻強壓雲豹
Date: 2025/03/02
Content: 儘管新北中信特攻奈森第三節才打25秒就告犯滿畢業，飛馳卻照樣可以狂飛到比賽終場，靠著飛馳全場37分，其中末節包辦21分，馬可也有36分，謝亞軒投進8顆三分球、攻下30分，特攻最後以116比106力退地主台啤永豐雲豹，也縮小兩隊之間勝差到0.5場。

地主雲豹雖然開賽就打出12比2攻勢，特攻卻靠謝亞軒的三分外線，以及馬可的內線單打逐漸追上，打到上半場還剩兩分多鐘，更取得44比42首度超前，不過奈森提前吞下第4犯，打完上半場，特攻仍以51比48領先，馬可半場就拿17分、7籃板。

第三節才打25秒，奈森被吹防守犯規，又因敲地板發洩被吹技術犯規，直接犯滿離場，僅剩兩名洋將可用的特攻，反而繼續保持領先優勢，來到末節，更靠飛馳、馬可兩人聯手擴大領先差距到9分

category: 體育
Title: MLB》勞資對抗再起？大聯盟預計2026年球季後封館
Date: 2025/03/02
Content: 大聯盟現行勞資協議將於2026年12月1日到期，預計球隊老闆屆時會封館停工以取得談判優勢，最壞的情況是殃及2027年球季。球員工會領袖克拉克（Tony Clark）本周對美媒說：「除非是我弄錯了，否則聯盟將會出來宣布停工，我相信我的說法是有憑有據的。」
在北美四大職業運動聯盟裡，棒球工會被認為立場最強硬、實力最雄厚，多次正面槓上資方，至今也只有職棒球員享有育嬰假（大谷翔平預計在新球季受益）。幾年前全球疫情爆發期間，大聯盟和資方耗費很多時間才說服工會接受減薪重開例行賽（縮水為60場）。
球員工會不喜歡大聯盟總裁曼佛雷德（Rob Manfred），曼佛雷德近年提出的幾項規則修改包括擴充季後賽規模、加入投球計時器、取締投手使用外部物質，被批評是危害球員健康。對於可能的封館情況，曼佛雷德今年初還正面看待，「從比較怪異的角度來說，這是一件好事。」
大聯盟相信封館給雙方帶來的壓力有助於加速談判過程（老闆和球員都沒有收入）。當然克拉克不這麼想，他批評說：「球員們都有親身經歷，他們知道這不是好事，停工被用來作為資方的武器，藉以威脅球員和他們的家庭。」


category: 體育
Title: 拳擊》泰森歷史最強一夜？把奧運金牌揍得不成人形
Date: 2025/03/02
Content: 退役拳王「鐵拳」麥克泰森（Mike Tyson）全盛期距離今日已有30多年，他的對手至今還是餘悸猶存，奧運金牌拳手畢格斯（Tyrell Biggs）最近回憶說：「我認為那晚的泰森可以打敗任人，包括賀利菲德、路易斯都會被他擊倒，歷史上不管任何人都難以對付那晚的泰森。」
1987年10月16日的三大聯盟統一冠軍戰，身高196公分、排名第1的挑戰者畢格斯在台上被泰森痛揍7個回合，最後裁判宣告終止比賽的時候，畢格斯的眉骨、鼻樑、臉頰、嘴唇都掛了彩，媒體形容「像是被丟進了榨汁機」。畢格斯是1982年世錦賽超重量級金牌、1984年洛杉磯奧運超重量級金牌，泰森則是連奧運都打不進去，1984年選拔賽輸給同胞選手提爾曼（Henry Tillman）。
畢格斯當時聲稱「我挨過比泰森更重的拳」「再打一次我能贏」，然而他從此脫離了頂級聯盟的競爭圈，那晚被拳壇公認是「永久改變

category: 運勢
Title: 3生肖3月福氣臨門 屬狗賺錢機會多
Date: 2025/03/02
Content: 3月已經來臨，命理專欄「十二生肖運勢」分享，生肖屬羊、狗和豬的人在3月份能揮別霉運、迎接好運，其中屬羊人貴人運旺盛，投資理財容易賺到錢；屬狗人賺錢機會多，還有月老幫忙牽紅線；屬豬人喜事一件接一件，單身者有望迎接新戀情。
▎生肖羊
屬羊人迎來「三合貴人」的吉運，貴人運非常旺盛，無論事業或生活都能成為助力；財運逐漸回升，對錢的敏感度也提高了，眼光放得更長遠，投資理財更容易賺到錢。感情運不錯，單身羊魅力大增，異性緣佳，這時主動出擊的成功率極高；正在熱戀中的羊則和另一半相處融洽，有望修成正果，步入婚姻殿堂。
▎生肖狗
屬狗人迎來「六合貴人」的吉運，財運尤其旺盛，賺錢機會多得數不清，以前借出的錢也能順利收回來，還有貴人帶來不少財源，求財計畫能夠順利進行。感情方面有「咸池」桃花入命，有月老幫忙牽紅線，單身狗身邊會出現心動對象，這時只要大方表白，對方有很高機率也在暗戀你。
▎生肖豬
屬豬人迎來「三合貴人」的吉運，工作、生活上的喜事一件接一件，且貴人運旺盛，財運滾滾，很容易有大筆錢財入帳。自主經營的屬豬人聚財能力強，有新的賺錢機會，只要抓住就能發大財；上班族的屬豬人思維敏捷、工作效率高，能得到更多的報酬。感情運勢表現也不錯，單身豬有望遇到一見鍾情的心儀對象，很快就能揮別單身。
★最近財運走旺？哪裡會有飛來桃花？快上《中時新聞網》LINE，鍵入自己的「星座」名稱或「運勢」搜尋。
★《中時新聞網》提醒您：命理、占星等說法僅供參考，請勿過度迷信。

category: 運勢
Title: 10種旺財風水一次看！廚房櫃子不能空
Date: 2025/03/02
Content: 在風水學中，住家環境有很多「眉角」要注意，任何細節都可能影響居住者的運勢或健康。《搜狐網》命理專欄指出，有10種能幫助聚財、旺財的風水，比如在玄關處擺屏風，能擋住「穿堂煞」；廚房的冰箱、櫃子必須保持半滿狀態；住家擺放綠色植物等。
▎玄關擺屏風
住家的玄關是藏風聚氣之地，如果大門正對窗戶，會形成風水上的「穿堂煞」，容易留不住錢財，建議可以在玄關擺放屏風、盆栽、簾子等，都可以有效阻擋不斷往外流失的錢財，從而達到聚財的目的。
▎門上貼福字
在進出的門上貼福字，寓意福到運到，招財納福，對納吉氣、招

category: 寶島
Title: 引入AI烘焙！桃園復興咖啡農打造羅馬廊道職人學苑
Date: 2025/03/02
Content: 桃園市復興區近年來推廣咖啡農產，同時也在國際評鑑賽事上獲得佳績，在地咖啡農陳耑妤打造「羅馬廊道職人學苑」，不僅提供在地青農完善的學習場域，未來也會開設各類手作課程傳承原住民文化，值得一提的是更引入AI烘焙機器，運用現代科技省時省力喝到復興好咖啡。

桃園市復興區除了以水蜜桃、甜柿、柑橘、綠竹筍等優質農產聞名全國外，自2020年開始積極推廣咖啡產業至今，已創立2班咖啡產銷班，種植面積達20公頃以上，復興區公所也扶植咖啡農，以小農栽種、精品咖啡為目標，各咖啡莊園在羅馬公路形成「羅馬咖啡廊道」。

陳耑妤是返鄉青農並鑽研咖啡豆種植及後製相關技術，她說，復興區有越來越多的咖啡農與各職類的職人，許多活動與資源集中在台七線，然而羅馬公路的咖啡廊道已經成形並且逐漸成熟，發現很多人需要往外尋求機會參與市集擺攤，或是租借咖啡廳等場域課程教學，起心動念希望創建平台，提供培訓及交流學習的空間外，也可以打造體驗課程吸引更多遊客創造經濟效益。

她也說，引進融合AI科技技術的咖啡後製機器，可以從事咖啡豆烘焙、挑豆、沖煮及杯側等環節，期盼科技技術成為助手而非取代人力，未來羅馬廊道職人學苑將會是咖啡專業培訓場域，也會不定期辦理編織、芳療、茶葉、飲食、原民傳統文化技藝等課程，除了推動地方創生以外，也鼓勵更多青農或是職人返鄉。

category: 寶島
Title: 嘉義抓周結合科技 「AR萌虎帽」讓親子同樂
Date: 2025/03/02
Content: 嘉義市「城隍佑民‧文昌護子」古禮抓周暨學童開筆禮活動，2日在嘉義市政府舉辦，40組家庭參加，滿周歲的寶寶們體驗人生第1個「小小成年禮」，還結合AR科技虎帽及古文物導覽解說圖文，讓大人小孩都體驗新科技的趣味。

曾參與第1屆抓周的幼兒今年即將入學讀小一，市長黃敏惠特地為他們點硃砂、發送限量的蛇年開運小紅包，祝福小寶貝們「蛇麼都如意、蛇麼都順心」。

黃敏惠表示，公益古禮抓周暨開筆禮儀式今年第7屆，有超過250名孩子受城隍爺與文昌帝君庇佑成長茁壯，還有文化傳承的意義，促進家庭及親子關係。

黃敏惠還提到，此活動融合科技推出「Q萌虎頭帽濾鏡」，是市府社會處、建設處與工策會共同輔導店家「嘉義市風格店家數位優化

category: 寶島
Title: 車城鄉有牙醫了！新衛生所「兼具多功能」每周4診
Date: 2025/03/02
Content: 屏東縣政府斥資縣款1.3億元打造的車城鄉衛生所2日落成啟用，由於是總統府祕書長潘孟安在任職屏東縣長時規畫，所以他特別出席啟用典禮，他說，社區醫療在偏鄉很重要，衛生所有如小型區域醫院般的存在，縣長周春米也說，車城鄉沒有牙醫診所，所以新建的衛生所還增設鄉內唯一牙科門診，每周4診服務鄉親。

原車城鄉衛生所在歷經40多年的使用後，在2年半前原址重建，2日落成啟用時，不僅許多民眾到場見證，當初爭取重建的潘孟安也到場同樂，他說，他當縣長時遇上疫情，那時他發現屏東很多衛生所在疫情期間都有空間與設施不足的問題，所以他都很積極爭取重建衛生所。

潘孟安笑說，長輩們都很聽衛生所主任的話，這也代表著社區醫療的信賴度與依存度，社區醫療在偏鄉真的很重要，所以在遲遲等不到中央補助的情況下，幸有縣議會的支持，同意全額撥縣款1.3億元打造有如小型區域醫院的全新車城衛生所。

對此，周春米很感謝潘孟安在任內有此規畫，她說，重建的車城衛生所不僅是衛生所，還同時兼備日照中心、心理衛生中心的功能，讓民眾可以檢查、運動兼諮詢，而最重要的是增設了牙醫門診，車城鄉沒有牙醫診所，這是全鄉唯一的牙醫門診，診間設有高壓消毒鍋、紫外線滅菌箱等設備，能提供完善的治療與診斷，每周有4診可服務鄉親。

她強調，車城鄉衛生所將提供車城及周邊鄉親更全面的健康照護，實踐衛生所的功能轉型，也向偏鄉醫療平權邁進一大步。

屏東衛生局表示，車城鄉老年人口比例在今年1月達25.37％，推估失能失智人口數約464人，為提升長照服務可近性，並強化在地網絡合作，衛生所2樓設有「車城照管分站」，提供便民服務平台，讓車城鄉民眾能即時獲得照顧諮詢與協助，而日照中心可服務29人，採失能身障混合營運方式，將導入多元的健康促進課程，如音樂輔療、積木輔療、園藝輔療，並透過專業復能自立功能訓練，讓失能者有再次自我實現的機會。

category: 寶島
Title: 汐止文化櫻花季 櫻花攝影展即日起展出
Date: 2025/03/02
Content: 為提升汐止文化櫻花季觀光效益，新北市汐止區公所與汐止區農會共同主辦 2025「文化櫻花」攝影比賽，共883件投稿參賽，歷經5輪評選，最終產生金牌1名、銀牌2名、銅牌3名

In [6]:
csv_file = 'chinatimes_' + now + '.csv'
file_path = "C:/Users/Administrator/INTERN/Chinatimes_news" #modify to your store_path 
csv_file_path = os.path.join(file_path, csv_file)

with open(csv_file_path, mode='w', newline='', encoding='utf-8-sig') as file:
    fieldnames = ['Link', 'Category', 'Title', 'Publish_time', 'Text']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    writer.writeheader()
    for news_data in all_news_data:
        if news_data:
            writer.writerow(news_data)

driver.quit()

In [7]:
driver.quit()